In [24]:
!pip install langchain_core langchain_community langchain_experimental langchain_openai neo4j yfiles_jupyter_graphs python-dotenv

In [25]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


In [26]:
from google.colab import userdata
api_key = userdata.get('AZURE_OPENAI_API_KEY')
azure_endpoint = userdata.get('AZURE_OPENAI_ENDPOINT')

In [27]:
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI


load_dotenv()

# api_key = os.getenv("AZURE_OPENAI_API_KEY")
# azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2023-07-01-preview"

llm = AzureChatOpenAI(
    model="gpt-4",
    azure_deployment="gpt-4",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
)

from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
)

llm_transformer = LLMGraphTransformer(llm=llm)

In [28]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm.invoke([message])

AIMessage(content="J'aime la programmation.", response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 19, 'total_tokens': 26}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_6d6abe0201', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-56b0a9c9-e632-4350-b759-551c895581

In [29]:
#retrieving environmental variables from the .env file
load_dotenv()

#alternative
os.environ["NEO4J_URI"] = "xxx"
os.environ["NEO4J_USERNAME"] = "xxx"
os.environ["NEO4J_PASSWORD"] = "xxx"

graph = Neo4jGraph()

## Exploring AuraDB: Test Harry Potter

In [30]:
from langchain_core.documents import Document

text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 9 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [ ]:
graph.add_graph_documents(
  graph_documents,
  baseEntityLabel=True,
  include_source=True
)

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

### Adding embeddings

In [20]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
)

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
query = "Who is Dudley?"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)


text: 
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for sever

### CypherChain

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is Mr. Dursley's job?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Mr. Dursley"})-[:WORKS_AT]->(o:Organization) RETURN o.id
Full Context:
[{'o.id': 'Grunnings'}]

> Finished chain.


{'query': "What is Mr. Dursley's job?",
 'result': 'Mr. Dursley works at Grunnings.'}

### QAChain

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever(), verbose = True
)

result = qa_chain({"query": "What is Mr. Dursley's job?"})
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'Mr. Dursley is the director of a firm called Grunnings, which makes drills.'

## GraphRAG

In [ ]:
raw_documents = WikipediaLoader(query="French Revolution", load_max_docs = 10).load()

In [ ]:
"""
from langchain_community.document_loaders import HuggingFaceDatasetLoader
dataset_name = "evidence_infer_treatment"
page_content_column = "Text"

#loading only the first 100 rows of the dataset
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column, name = '2.0', )
"""

In [ ]:
len(raw_documents)

10

In [ ]:
raw_documents

[Document(page_content="The French Revolution was a period of political and societal change in France that began with the Estates General of 1789, and ended with the coup of 18 Brumaire in November 1799 and the formation of the French Consulate. Many of its ideas are considered fundamental principles of liberal democracy, while its values and institutions remain central to modern French political discourse.\nIts causes are generally agreed to be a combination of social, political and economic factors, which the Ancien Régime proved unable to manage. A financial crisis and widespread social distress led in May 1789 to the convocation of the Estates General, which was converted into a National Assembly in June. The Storming of the Bastille on 14 July led to a series of radical measures by the Assembly, among them the abolition of feudalism, state control over the Catholic Church in France, and a declaration of rights.\nThe next three years were dominated by the struggle for political con

In [ ]:
for i in range(len(raw_documents)):
    print(f'Document {i}')
    print(raw_documents[i].page_content)

Document 0
The French Revolution was a period of political and societal change in France that began with the Estates General of 1789, and ended with the coup of 18 Brumaire in November 1799 and the formation of the French Consulate. Many of its ideas are considered fundamental principles of liberal democracy, while its values and institutions remain central to modern French political discourse.
Its causes are generally agreed to be a combination of social, political and economic factors, which the Ancien Régime proved unable to manage. A financial crisis and widespread social distress led in May 1789 to the convocation of the Estates General, which was converted into a National Assembly in June. The Storming of the Bastille on 14 July led to a series of radical measures by the Assembly, among them the abolition of feudalism, state control over the Catholic Church in France, and a declaration of rights.
The next three years were dominated by the struggle for political control, exacerbat

In [ ]:
from langchain_core.documents import Document
#attributing an empty dict to the metadata field of each document
raw_documents = [Document(page_content=doc.page_content, metadata={}) for doc in raw_documents]
raw_documents[0].metadata

{}

In [ ]:
# Define chunking strategy

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
import pickle

# Save the documents variable
with open('documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

## Storing documents in a GraphDB

In [ ]:
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
from langchain.chains import RetrievalQA

qa_graph_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever(), verbose = True
)

result = qa_graph_chain({"query": "Considering the financial crisis and resistance to reform by the Ancien Régime, how did the convocation of the Estates General in May 1789 specifically address these multifaceted issues?"})
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'The convocation of the Estates General in May 1789 was an attempt to address the financial crisis and other issues plaguing France. However, it did not immediately resolve these problems. The Estates General was a legislative assembly of the different classes (or estates) of French subjects. It had not been convened for 175 years prior to 1789, indicating the severity of the crisis at hand.\n\nThe calling of the Estates General was intended to be a way for the monarchy to discuss and approve new tax laws to resolve the financial crisis. It provided a platform for the three estates - the clergy, the nobility, and the commoners - to voice their grievances and propose solutions.\n\nHowever, the Estates General was not equipped to handle the depth of the crisis due to its structure and the conflicts between the estates. The Third Estate, representing the commoners who were the majority of the population and bore the brunt of the tax burden, was outvoted by the other two estates. This led 

# Graph only

In [ ]:
# Defining each question as a variable
question1 = "How did the economic policies of the Ancien Régime contribute to the financial crisis that precipitated the French Revolution?"
question2 = "In what ways did the social and political structure of the Estates-General contribute to its transformation into the National Assembly?"
question3 = "What role did economic depression and military defeats play in the radicalization of the French Revolution in 1792?"
question4 = "How did the French Revolutionary Wars affect the internal political landscape of France from 1792 to 1799?"
question5 = "Examine the socio-economic reasons behind the calling of the Estates-General in 1789."
question6 = "How did Enlightenment ideas influence the legislative reforms of the National Assembly?"
question7 = "What event directly led to the transformation of the Estates-General into the National Assembly in June 1789?"
question8 = "Which radical measure taken by the National Assembly on July 14, 1789, symbolically marked the beginning of the French Revolution?"
question9 = "Which governing body replaced the National Convention after the fall of Robespierre in 1794?"
question10 = "What significant political change occurred in France on 18 Brumaire in 1799?"
question11 = "Considering the financial difficulties faced by the Ancien Régime, how did the complex and inconsistent tax system contribute to the financial instability and eventual calling of the Estates-General?"
question12 = "What role did the socio-economic pressures such as the increase in the population and the widening gap between the rich and the poor play in setting the stage for the French Revolution?"
question13 = "How did the financial crisis, exacerbated by poor harvests and high food prices, lead to the convening of the Estates-General in 1789?"
question14 = "Discuss the immediate political repercussions of the Storming of the Bastille on the French Revolution."

# Creating a list of all questions
questions = [question1, question2, question3, question4, question5, question6, question7, question8, question9, question10, question11, question12, question13, question14]




In [ ]:
graph_results = []
graph_source_documents = []
for q in questions:
    graph_results.append(graph_chain({"query": q})["result"])
    graph_source_documents.append(graph_chain({"query": q})["source_documents"])



In [ ]:
import pandas as pd

# Assuming questions, rag_results, graph_results are your lists
df = pd.DataFrame({
    'questions': questions,
    'graph_results': graph_results,
    'graph_source_documents': graph_source_documents
})

df.head(20)

,questions,graph_results,graph_source_documents
0,How did the economic policies of the Ancien Ré...,The economic policies of the Ancien Régime con...,"[page_content=""\ntext: % of the population by ..."
1,In what ways did the social and political stru...,The social and political structure of the Esta...,[page_content='\ntext: The French Revolution w...
2,How did the Storming of the Bastille influence...,The text does not provide specific information...,[page_content='\ntext: introduction of the pr...
3,What role did economic depression and military...,Economic depression and military defeats playe...,"[page_content=""\ntext: % of the population by ..."
4,Discuss the impact of the Reign of Terror on t...,The text does not provide specific information...,[page_content='\ntext: introduction of the pr...
5,What were the primary factors leading to the f...,The text does not provide specific information...,[page_content='\ntext: The French Revolution w...
6,How did the French Revolutionary Wars affect t...,The French Revolutionary Wars that began in 17...,[page_content='\ntext: The French Revolution w...
7,Examine the socio-economic reasons behind the ...,The calling of the Estates-General in 1789 was...,"[page_content=""\ntext: % of the population by ..."
8,How did Enlightenment ideas influence the legi...,The Enlightenment ideas played a crucial role ...,[page_content='\ntext: introduction of the pr...
9,Assess the role of Louis XVI’s indecisiveness ...,The text does not provide specific information...,[page_content='\ntext: rituals with a detaile...


In [ ]:
graphrag_text = []
for index, row in df.iterrows():
    graphrag_documents = row['graph_source_documents']
    combined_graphrag_documents = " ".join(doc.page_content for doc in graphrag_documents)
    graphrag_text.append(combined_graphrag_documents)


df.graph_source_documents = graphrag_text
df.head(20)

,questions,graph_results,graph_source_documents
0,How did the economic policies of the Ancien Ré...,The economic policies of the Ancien Régime con...,\ntext: % of the population by 1789. Although ...
1,In what ways did the social and political stru...,The social and political structure of the Esta...,\ntext: The French Revolution was a period of ...
2,What role did economic depression and military...,Economic depression and military defeats playe...,\ntext: % of the population by 1789. Although ...
3,How did the French Revolutionary Wars affect t...,The French Revolutionary Wars that began in 17...,\ntext: The French Revolution was a period of ...
4,Examine the socio-economic reasons behind the ...,The calling of the Estates-General in 1789 was...,\ntext: % of the population by 1789. Although ...
5,How did Enlightenment ideas influence the legi...,The Enlightenment ideas played a crucial role ...,\ntext: introduction of the principle of lega...
6,What event directly led to the transformation ...,"On June 17, 1789, the deputies of the Third Es...",\ntext: for delegates to the Estates-General\...
7,Which radical measure taken by the National As...,"The Storming of the Bastille on July 14, 1789,...",\ntext: The French Revolution was a period of ...
8,Which governing body replaced the National Con...,The governing body that replaced the National ...,\ntext: The French Revolution of 1848 (French:...
9,What significant political change occurred in ...,The significant political change that occurred...,\ntext: The French Revolution of 1848 (French:...


In [ ]:
print('QUESTION')
print(df.iloc[0]['questions'])
#printing whitespace
print('')
print('ANSWER')
print(df.iloc[0]['graph_results'])
print('')
print('CONTEXT')
print(df.iloc[0]['graph_source_documents'])

QUESTION
How did the economic policies of the Ancien Régime contribute to the financial crisis that precipitated the French Revolution?

ANSWER
The economic policies of the Ancien Régime contributed to the financial crisis that precipitated the French Revolution in several ways. Firstly, the state had a high level of debt, although it was not high compared to Britain's. The tax system was a major issue, with tax rates varying widely from one region to another, often different from the official amounts, and collected inconsistently. This complexity created uncertainty over the amount any authorised tax actually contributed, and caused resentment among taxpayers.

Attempts to simplify the system were blocked by the regional Parlements, which had the power to approve financial policy. This led to a financial impasse that contributed to the calling of the Estates-General, a representative assembly of the three estates (clergy, nobility, and commoners), which then became radicalised by the 

# Evaluation metrics

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

groundedness_critique_prompt = PromptTemplate.from_template("""
You will be given a context and answer about that context.
Your task is to provide a 'total rating' scoring how well the ANSWER is entailed by the CONTEXT.
Give your answer on a scale of 1 to 5, where 1 means that the ANSWER is logically false from the information contained in the CONTEXT, and 5 means that the ANSWER follows logically from the information contained in the CONTEXT.

Provide your response in a list as follows:

Response:::
[Evaluation: (your rationale for the rating, as a text),
Total rating: (your rating, as a number between 1 and 5)]

You MUST provide values for 'Evaluation:' and 'Total rating:' in your response.

Now here are the context, question and answer.

Context: {context}\n
Answer: {answer}\n
Response::: """)

relevance_critique_prompt = PromptTemplate.from_template("""
You will be given a context, and question and answer about that context.
Your task is to provide a 'total rating' to measure how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.
Given the context and question, score the relevance of the answer between one to five stars using the following rating scale:

Give your response on a scale of 1 to 5, where 1 means that the answer doesn't address the question at all, and 5 means that the answer is perfectly matching the question.

Provide your response in a list as follows:

Response:::
[Evaluation: (your rationale for the rating, as a text),
Total rating: (your rating, as a number between 1 and 5)]

You MUST provide values for 'Evaluation:' and 'Total rating:' in your response.

Now here is the question.

Answer: {answer}\n
Question: {question}\n
Context: {context}\n
Response::: """)

coherence_critique_prompt = PromptTemplate.from_template("""
You will be given a question and answer.
Your task is to measure the coherence of the answer. Coherence is measured by how well all the sentences fit together and sound naturally as a whole. Consider the overall quality of the answer when evaluating coherence.
Given the question and answer, score the coherence of answer on a scale of 1 to 5, where 1 means that the answer completely lacks coherence, 5 means that the answer has perfect coherency.

Provide your response in a list as follows:

Response:::
[Evaluation: (your rationale for the rating, as a text),
Total rating: (your rating, as a number between 1 and 5)]

You MUST provide values for 'Evaluation:' and 'Total rating:' in your response.

Now here is the question.

Question: {question}\n
Answer: {answer}\n
Response::: """)

## Graph evaluation

In [ ]:
groundness = []
relevance = []
coherence = []
for i in range(len(df)):
    question = df.iloc[i]['questions']
    answer = df.iloc[i]['graph_results']
    context = df.iloc[i]['graph_source_documents']
    groundness_chain = LLMChain(llm=llm, prompt=groundedness_critique_prompt)
    groundness.append(groundness_chain.run(context=context, answer = answer))
    relevance_chain = LLMChain(llm=llm, prompt=relevance_critique_prompt)
    relevance.append(relevance_chain.run(question=question, answer = answer, context = context))
    coherence_chain = LLMChain(llm=llm, prompt=coherence_critique_prompt)
    coherence.append(coherence_chain.run(question=question, answer = answer))


In [ ]:
#adding the three lists groundness, relevance and standalone to the dataframe
df['groundness'] = groundness
df['relevance'] = relevance
df['coherence'] = coherence
df.head()

,questions,graph_results,graph_source_documents,groundness,relevance,coherence
0,How did the economic policies of the Ancien Ré...,The economic policies of the Ancien Régime con...,\ntext: % of the population by 1789. Although ...,Evaluation: The ANSWER correctly identifies se...,[Evaluation: The answer provides a comprehensi...,[Evaluation: The answer demonstrates a high le...
1,In what ways did the social and political stru...,The social and political structure of the Esta...,\ntext: The French Revolution was a period of ...,[Evaluation: The ANSWER accurately reflects th...,[Evaluation: The answer thoroughly addresses t...,[Evaluation: The answer is well-structured and...
2,What role did economic depression and military...,Economic depression and military defeats playe...,\ntext: % of the population by 1789. Although ...,[Evaluation: The ANSWER is well supported by t...,[Evaluation: The answer provides a comprehensi...,[Evaluation: The answer is coherent and presen...
3,How did the French Revolutionary Wars affect t...,The French Revolutionary Wars that began in 17...,\ntext: The French Revolution was a period of ...,[Evaluation: The ANSWER provides a summary of ...,[Evaluation: The answer provided is comprehens...,"[Evaluation: The answer is coherent, providing..."
4,Examine the socio-economic reasons behind the ...,The calling of the Estates-General in 1789 was...,\ntext: % of the population by 1789. Although ...,[Evaluation: The answer aligns closely with th...,[Evaluation: The answer provides a comprehensi...,[Evaluation: The answer presents a coherent na...


In [ ]:
print(df.iloc[0]['relevance'])

[Evaluation: The answer provides a comprehensive overview of how the economic policies of the Ancien Régime contributed to the financial crisis that led to the French Revolution. It addresses the state's high level of debt, the inconsistent and complex tax system, the resistance to reform by regional Parlements, the economic recession, and the bad harvests that led to high unemployment and food prices. The answer also situates these factors in the broader context of resistance to reform by the ruling elite and indecision by Louis XVI. All of these points reflect the significant aspects mentioned in the provided context regarding the economic policies and their contribution to the crisis. Therefore, the answer is well-aligned with the context and the question asked.
Total rating: 5]


In [ ]:
#printing the content of each column in the first row of the dataframe, separating the name of the column from the content of the cell
print('QUESTION')
print(df.iloc[0]['questions'])
#printing whitespace
print('')
print('ANSWER')
print(df.iloc[0]['graph_results'])
print('')
print('CONTEXT')
print(df.iloc[0]['graph_source_documents'])
print('')
print('GROUNDNESS')
print(df.iloc[0]['groundness'])
print('')
print('RELEVANCE')
print(df.iloc[0]['relevance'])
print('')
print('STANDALONE')
print(df.iloc[0]['standalone'])

QUESTION
How did the economic policies of the Ancien Régime contribute to the financial crisis that precipitated the French Revolution?

ANSWER
The economic policies of the Ancien Régime contributed to the financial crisis that precipitated the French Revolution in several ways. Firstly, the state had a high level of debt, although it was not high compared to Britain's. The tax system was a major issue, with tax rates varying widely from one region to another, often different from the official amounts, and collected inconsistently. This complexity created uncertainty over the amount any authorised tax actually contributed, and caused resentment among taxpayers.

Attempts to simplify the system were blocked by the regional Parlements, which had the power to approve financial policy. This led to a financial impasse that contributed to the calling of the Estates-General, a representative assembly of the three estates (clergy, nobility, and commoners), which then became radicalised by the 

In [ ]:
# Save the DataFrame to an Excel file
df.to_excel('graphrag_results.xlsx', index=False)